# 라이브러리 불러오기


In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


In [ ]:
# 분석에 필요한 library 불러오기
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from datetime import datetime

In [ ]:
# 구글드라이브 연결
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# 데이터 불러오기

In [ ]:
# user_spec
user_spec = pd.read_csv("/content/gdrive/MyDrive/2022빅콘테스트_데이터분석리그_데이터분석분야_퓨처스부문_데이터셋_220908/user_spec.csv")

In [ ]:
# loan_result
loan_result = pd.read_csv('/content/gdrive/MyDrive/2022빅콘테스트_데이터분석리그_데이터분석분야_퓨처스부문_데이터셋_220908/loan_result.csv')

## result table에는 없는 user_spec정보 삭제
- 기간차이로 삭제되었거나 대출승인을 받지 못한 경우

In [ ]:
user_spec.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1394216 entries, 0 to 1394215
Data columns (total 17 columns):
 #   Column                               Non-Null Count    Dtype  
---  ------                               --------------    -----  
 0   application_id                       1394216 non-null  int64  
 1   user_id                              1394216 non-null  int64  
 2   birth_year                           1381255 non-null  float64
 3   gender                               1381255 non-null  float64
 4   insert_time                          1394216 non-null  object 
 5   credit_score                         1289101 non-null  float64
 6   yearly_income                        1394126 non-null  float64
 7   income_type                          1394131 non-null  object 
 8   company_enter_month                  1222456 non-null  float64
 9   employment_type                      1394131 non-null  object 
 10  houseown_type                        1394131 non-null  object 
 11

In [ ]:
# user_id와 loan_limit값이 nan인 경우의 행 삭제
loan_result = loan_result.dropna(subset=['loan_limit'])

In [ ]:
# 중복된 application_id+product_id 제거하기

loan_result=loan_result.drop_duplicates(['application_id', 'product_id'], keep = False)

In [ ]:
# 모두 거절되거나 중간에 이탈한 경우
app_no_result = list(set(user_spec['application_id'].unique()) - set(loan_result['application_id'].unique()))
user_spec = user_spec[user_spec['application_id'].isin(app_no_result)==False]
user_spec.shape

(968693, 17)

In [ ]:
# 기간 차이로 인해 누락된 데이터로, 해당 데이터는 제외
result_no_app = list(set(loan_result['application_id'].unique()) - set(user_spec['application_id'].unique()))
loan_result = loan_result[loan_result['application_id'].isin(result_no_app)==False]
loan_result.shape

(13518980, 7)

In [ ]:
result_no_app = list(set(loan_result['application_id'].unique()) - set(user_spec['application_id'].unique()))
result_no_app

[]

# 변수 수정하기

In [ ]:
# purpose 영어를 한국어로 변경
user_spec.loc[(user_spec['purpose']=='LIVING'),'purpose'] = '생활비' 
user_spec.loc[(user_spec['purpose']=='SWITCHLOAN'),'purpose'] = '대환대출'
user_spec.loc[(user_spec['purpose']=='BUSINESS'),'purpose'] = '사업자금'
user_spec.loc[(user_spec['purpose']=='HOUSEDEPOSIT'),'purpose'] = '전월세보증금'
user_spec.loc[(user_spec['purpose']=='ETC'),'purpose'] = '기타'
user_spec.loc[(user_spec['purpose']=='BUYHOUSE'),'purpose'] = '주택구입'
user_spec.loc[(user_spec['purpose']=='INVEST'),'purpose'] = '투자'
user_spec.loc[(user_spec['purpose']=='BUYCAR'),'purpose'] = '자동차구입'

In [ ]:
user_spec['purpose'].value_counts(dropna = False)

생활비       578937
대환대출      267447
사업자금       46460
전월세보증금     22551
기타         22437
주택구입       16720
투자         11240
자동차구입       2901
Name: purpose, dtype: int64

In [ ]:
user_spec.isna().sum()

application_id                              0
user_id                                     0
birth_year                               8592
gender                                   8592
insert_time                                 0
credit_score                            81767
yearly_income                               1
income_type                                 0
company_enter_month                     92304
employment_type                             0
houseown_type                               0
desired_amount                              0
purpose                                     0
personal_rehabilitation_yn             417738
personal_rehabilitation_complete_yn    843936
existing_loan_cnt                      146287
existing_loan_amt                      225037
dtype: int64

# train-test 나누기 

In [ ]:
train_app = list(set(loan_result.loc[loan_result['is_applied'].notna(),'application_id']))
test_app = list(set(loan_result.loc[loan_result['is_applied'].isna(),'application_id']))

In [ ]:
train_user = user_spec[user_spec['application_id'].isin(train_app)]
test_user = user_spec[user_spec['application_id'].isin(test_app)]

In [ ]:
train_result = loan_result[loan_result['is_applied'].notna()]
test_result = loan_result[loan_result['is_applied'].isna()]

# 변수 이상치/ 결측치 + 파생변수

In [ ]:
# 결측치 수 파생변수 생성
train_user['NA_count']=train_user.isna().sum(axis=1)
test_user['NA_count']=test_user.isna().sum(axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


## 개인회생자 + 납입완료

In [ ]:
##train

#개인회생자가 아닌데 납입완료(이상치) -> drop
no_user1=train_user[(train_user['personal_rehabilitation_yn']==0) & (train_user['personal_rehabilitation_complete_yn']==1)].index
train_user.drop(no_user1,inplace=True)

#개인회생자가 아닌 사람 -> 1
condition2 = train_user['personal_rehabilitation_yn']==0
train_user.loc[condition2, 'personal_rehabilitation_complete_yn']= '1'

#개인회생자중 납입중 -> 2
condition2 = train_user['personal_rehabilitation_complete_yn']==0
train_user.loc[condition2, 'personal_rehabilitation_complete_yn']= '2'

#개인회생자중 납입완료 -> 3
condition3 = train_user['personal_rehabilitation_complete_yn']==1
train_user.loc[condition3, 'personal_rehabilitation_complete_yn']= '3'

#개인회생자 버튼 건너뛰기(NULL값) -> 0
train_user['personal_rehabilitation_complete_yn'] = train_user['personal_rehabilitation_complete_yn'].fillna('0')

train_user['personal_rehabilitation_complete_yn'].value_counts()

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/panda

0    416630
1    309452
2      2372
3       378
Name: personal_rehabilitation_complete_yn, dtype: int64

In [ ]:
##test

#개인회생자가 아닌데 납입완료(이상치) -> drop
no_user1=test_user[(test_user['personal_rehabilitation_yn']==0) & (test_user['personal_rehabilitation_complete_yn']==1)].index
test_user.drop(no_user1,inplace=True)

#개인회생자가 아닌 사람 -> 1
condition2 = test_user['personal_rehabilitation_yn']==0
test_user.loc[condition2, 'personal_rehabilitation_complete_yn']= '1'

#개인회생자중 납입중 -> 2
condition2 = test_user['personal_rehabilitation_complete_yn']==0
test_user.loc[condition2, 'personal_rehabilitation_complete_yn']= '2'

#개인회생자중 납입완료 -> 3
condition3 = test_user['personal_rehabilitation_complete_yn']==1
test_user.loc[condition3, 'personal_rehabilitation_complete_yn']= '3'

#개인회생자 버튼 건너뛰기(NULL값) -> 0
test_user['personal_rehabilitation_complete_yn'] = test_user['personal_rehabilitation_complete_yn'].fillna('0')

test_user['personal_rehabilitation_complete_yn'].value_counts()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


1    236642
2      1851
0      1108
3       257
Name: personal_rehabilitation_complete_yn, dtype: int64

## birth_year & gender

In [ ]:
train_user_spec=train_user.groupby('user_id').agg(**{
    'birth_year':pd.NamedAgg(column='birth_year', aggfunc='mean'),
    'gender':pd.NamedAgg(column='gender', aggfunc='mean'),    
}).reset_index()

In [ ]:
## train 

train_user.drop(['birth_year', 'gender'], axis=1, inplace=True)
train_user = pd.merge(train_user, train_user_spec, on='user_id', how='left')

In [ ]:
train_user.isna().sum()

application_id                              0
user_id                                     0
insert_time                                 0
credit_score                            67120
yearly_income                               0
income_type                                 0
company_enter_month                     68413
employment_type                             0
houseown_type                               0
desired_amount                              0
purpose                                     0
personal_rehabilitation_yn             416630
personal_rehabilitation_complete_yn         0
existing_loan_cnt                      115774
existing_loan_amt                      174127
NA_count                                    0
birth_year                               4832
gender                                   4832
dtype: int64

In [ ]:
## test

test_user = pd.merge(test_user, train_user_spec, on='user_id', how='left')
test_user['gender'] = test_user[['gender_x','gender_y']].mean(axis=1)
test_user['birth_year'] = test_user[['birth_year_x','birth_year_y']].mean(axis=1)
test_user['gender'].value_counts(dropna=False)

1.0    159798
0.0     77956
NaN      2104
Name: gender, dtype: int64

In [ ]:
test_user.drop(['birth_year_x', 'birth_year_y', 'gender_x', 'gender_y'], axis=1, inplace=True)

In [ ]:
test_user.isna().sum()

application_id                             0
user_id                                    0
insert_time                                0
credit_score                           14646
yearly_income                              1
income_type                                0
company_enter_month                    23891
employment_type                            0
houseown_type                              0
desired_amount                             0
purpose                                    0
personal_rehabilitation_yn              1108
personal_rehabilitation_complete_yn        0
existing_loan_cnt                      30512
existing_loan_amt                      50908
NA_count                                   0
gender                                  2104
birth_year                              2104
dtype: int64

## age

In [ ]:
# age 파생변수 생성
def age(d1, d2):
  return (d1.year-d2+1)

## train
train_user['insert_time'] = pd.to_datetime(train_user['insert_time'])
train_user['age'] = train_user.apply(lambda x: age(x.insert_time, x.birth_year), axis=1)

In [ ]:
## test
test_user['insert_time'] = pd.to_datetime(test_user['insert_time'])
test_user['age'] = test_user.apply(lambda x: age(x.insert_time, x.birth_year), axis=1)

In [ ]:
# age 결측치 mean값으로 바꾸기
age_mean = round(train_user['age'].mean(),0)

## train 
train_user.loc[train_user['age'].isna(),'age'] = age_mean
## test 
test_user.loc[test_user['age'].isna(),'age'] = age_mean

In [ ]:
test_user['age']

0         55.0
1         26.0
2         34.0
3         23.0
4         47.0
          ... 
239853    30.0
239854    40.0
239855    37.0
239856    38.0
239857    40.0
Name: age, Length: 239858, dtype: float64

## existing_loan_cnt & existing_loan_amt

In [ ]:
# loan_cnt = 1 인데 loan_amt = nan 인 애들
loan_amt_null = train_user.existing_loan_amt.isna() == True
train_user.loc[loan_amt_null,'existing_loan_amt'] = 0

# loan_cnt가 nan인 애들 -> 마이데이터에
loan_cond = (train_user['existing_loan_cnt'].isna() == True)
train_user.loc[loan_cond,'existing_loan_cnt'] = 0
train_user.loc[loan_cond, 'existing_loan_amt'] = 0

In [ ]:
## test
# loan_cnt = 1 인데 loan_amt = nan 인 애들
loan_amt_null = test_user.existing_loan_amt.isna() == True
test_user.loc[loan_amt_null,'existing_loan_amt'] = 0

# loan_cnt가 nan인 애들 -> 마이데이터에
loan_cond = (test_user['existing_loan_cnt'].isna() == True)
test_user.loc[loan_cond,'existing_loan_cnt'] = 0
test_user.loc[loan_cond, 'existing_loan_amt'] = 0

In [ ]:
train_user.isna().sum()

application_id                              0
user_id                                     0
insert_time                                 0
credit_score                            67120
yearly_income                               0
income_type                                 0
company_enter_month                     68413
employment_type                             0
houseown_type                               0
desired_amount                              0
purpose                                     0
personal_rehabilitation_yn             416630
personal_rehabilitation_complete_yn         0
existing_loan_cnt                           0
existing_loan_amt                           0
NA_count                                    0
birth_year                               4832
gender                                   4832
age                                         0
dtype: int64

In [ ]:
test_user.isna().sum()

application_id                             0
user_id                                    0
insert_time                                0
credit_score                           14646
yearly_income                              1
income_type                                0
company_enter_month                    23891
employment_type                            0
houseown_type                              0
desired_amount                             0
purpose                                    0
personal_rehabilitation_yn              1108
personal_rehabilitation_complete_yn        0
existing_loan_cnt                          0
existing_loan_amt                          0
NA_count                                   0
gender                                  2104
birth_year                              2104
age                                        0
dtype: int64

## company_enter_month

In [ ]:
# train
train_user['company_enter_month'] = train_user['company_enter_month'].fillna(-1)
train_user['company_enter_month'] = train_user['company_enter_month'].astype(int).astype(str)
train_user['company_enter_month'] = train_user['company_enter_month'].apply(lambda x : x[:-2]  if len(x)>=8 else x)
train_user['company_enter_month'] = train_user['company_enter_month'].replace('-1',np.nan)
train_user['company_enter_month']= pd.to_datetime(train_user['company_enter_month'],format='%Y%m')

# test
test_user['company_enter_month'] = test_user['company_enter_month'].fillna(-1)
test_user['company_enter_month'] = test_user['company_enter_month'].astype(int).astype(str)
test_user['company_enter_month'] = test_user['company_enter_month'].apply(lambda x : x[:-2]  if len(x)>=8 else x)
test_user['company_enter_month'] = test_user['company_enter_month'].replace('-1',np.nan)
test_user['company_enter_month']= pd.to_datetime(test_user['company_enter_month'],format='%Y%m')

In [ ]:
train_user['insert_time']

0        2022-04-20 01:23:09
1        2022-04-20 00:38:18
2        2022-04-20 15:01:01
3        2022-04-20 15:21:40
4        2022-04-20 15:44:42
                 ...        
728827   2022-03-22 14:55:32
728828   2022-03-22 01:19:24
728829   2022-03-22 07:34:32
728830   2022-03-22 22:12:35
728831   2022-03-22 08:55:14
Name: insert_time, Length: 728832, dtype: datetime64[ns]

In [ ]:
train_user['company_enter_month']

0        2021-10-01
1        2018-03-01
2        2019-01-01
3        2022-04-01
4        2020-12-01
            ...    
728827   2021-06-01
728828          NaT
728829   2009-08-01
728830   2017-05-01
728831   2011-03-01
Name: company_enter_month, Length: 728832, dtype: datetime64[ns]

In [ ]:
#insert_time보다 company_enter_month가 더 최근인 사람 -> drop
weird=train_user[pd.to_datetime(train_user['insert_time'] ,format='%Y%m')<=train_user['company_enter_month']].index
train_user.drop(weird, inplace=True)

In [ ]:
# 입사한 월차로 보기
def diff_month(d1, d2):
    return (d1.year - d2.year) * 12 + d1.month - d2.month

# train
train_user['company_month'] = train_user.apply(lambda x: diff_month(x.insert_time, x.company_enter_month), axis=1)
# test
test_user['company_month'] = test_user.apply(lambda x: diff_month(x.insert_time, x.company_enter_month), axis=1)

In [ ]:
# company_enter_month가 nan인 경우는 대부분 직업이 OTHERINCOME이므로, 입사연월이 0이라고 생각하여 0이라고 예측한다

# train
train_user.loc[train_user['income_type']=='OTHERINCOME', 'company_enter_month']=0

#test
test_user.loc[test_user['income_type']=='OTHERINCOME', 'company_enter_month']=0

In [ ]:
# company_month nan인 경우는 대부분 직업이 OTHERINCOME이므로, 입사연월이 0이라고 생각하여 0이라고 예측한다

# train
train_user.loc[train_user['income_type']=='OTHERINCOME', 'company_month']=0

#test
test_user.loc[test_user['income_type']=='OTHERINCOME', 'company_month']=0

## company_month & yearly income결측치

In [ ]:
# 수입 타입이 OTHERINCOME인 경우, 평균값으로 대체
cm_mean = round(train_user['company_month'].mean(),0)

## train 
train_user.loc[train_user['company_month'].isna(),'company_month'] = cm_mean
## test 
test_user.loc[test_user['company_month'].isna(),'company_month'] = cm_mean

In [ ]:
# yearly_income NA값 평균값으로 대체
income_mean = round(train_user['yearly_income'].mean(),0)

## train 
train_user.loc[train_user['yearly_income'].isna(),'yearly_income'] = income_mean
## test 
test_user.loc[test_user['yearly_income'].isna(),'yearly_income'] = income_mean

In [ ]:
test_user.isna().sum()

application_id                             0
user_id                                    0
insert_time                                0
credit_score                           14646
yearly_income                              0
income_type                                0
company_enter_month                        1
employment_type                            0
houseown_type                              0
desired_amount                             0
purpose                                    0
personal_rehabilitation_yn              1108
personal_rehabilitation_complete_yn        0
existing_loan_cnt                          0
existing_loan_amt                          0
NA_count                                   0
gender                                  2104
birth_year                              2104
age                                        0
company_month                              0
dtype: int64

## 기대출과다자 변수 생성

In [ ]:
## train
condition = train_user['yearly_income']*1.5 < train_user['existing_loan_amt']
train_user['lotloan'] = 0
train_user.loc[condition, 'lotloan'] = 1

## test
condition = test_user['yearly_income']*1.5 < test_user['existing_loan_amt']
test_user['lotloan'] = 0
test_user.loc[condition, 'lotloan'] = 1

# user_sepc & loan_result 합치기

In [ ]:
train=pd.DataFrame.merge(train_user, train_result, on='application_id', how='right')
train.isna().sum()

In [ ]:
# 이상치에서 drop된 train data drop하기
train.dropna(subset=['user_id'], inplace=True)
train.isna().sum()

In [ ]:
test=pd.DataFrame.merge(test_user, test_result, on='application_id', how='right')
test.isna().sum()

# 파생변수 생성

## application개수에 따른 파생변수 생성

In [ ]:
count=train.groupby('user_id').agg(**{
    'application_count':pd.NamedAgg(column='application_id', aggfunc=lambda x:x.nunique()),
    'product_count':pd.NamedAgg(column='product_id', aggfunc='count'),
    'apply_sum':pd.NamedAgg(column='is_applied', aggfunc='sum')
}).reset_index()

In [ ]:
count['product_rate'] = round(count['product_count']/count['application_count'],3)
count['apply_rate'] = round(count['apply_sum']/count['product_count'],3)

In [ ]:
count.drop(['product_count', 'apply_sum'], axis=1,inplace=True)

In [ ]:
count

# 테스트

In [ ]:
df=pd.DataFrame.merge(count, train, on='user_id', how='right')
df.isna().sum()

user_id                                      0
application_count                            0
product_rate                                 0
apply_rate                                   0
application_id                               0
insert_time                                  0
credit_score                           1243238
yearly_income                                0
income_type                                  0
company_enter_month                         55
employment_type                              0
houseown_type                                0
desired_amount                               0
purpose                                      0
personal_rehabilitation_yn             5869929
personal_rehabilitation_complete_yn          0
existing_loan_cnt                            0
existing_loan_amt                            0
NA_count                                     0
birth_year                               77226
gender                                   77226
age          

In [ ]:
df.corr()

# 전처리한 데이터 저장하기

In [ ]:
import os
os.chdir("/content/gdrive/MyDrive/2022빅콘테스트_데이터분석리그_데이터분석분야_퓨처스부문_데이터셋_220908")

In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3255194 entries, 0 to 3255193
Data columns (total 27 columns):
 #   Column                               Dtype         
---  ------                               -----         
 0   application_id                       int64         
 1   user_id                              int64         
 2   insert_time                          datetime64[ns]
 3   credit_score                         float64       
 4   yearly_income                        float64       
 5   income_type                          object        
 6   company_enter_month                  object        
 7   employment_type                      object        
 8   houseown_type                        object        
 9   desired_amount                       float64       
 10  purpose                              object        
 11  personal_rehabilitation_yn           float64       
 12  personal_rehabilitation_complete_yn  object        
 13  existing_loan_cnt          

In [ ]:
df.to_csv('./train.csv', encoding = 'cp949')

In [ ]:
test.to_csv('./test.csv', encoding = 'cp949')

In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3255194 entries, 0 to 3255193
Data columns (total 27 columns):
 #   Column                               Dtype         
---  ------                               -----         
 0   application_id                       int64         
 1   user_id                              int64         
 2   insert_time                          datetime64[ns]
 3   credit_score                         float64       
 4   yearly_income                        float64       
 5   income_type                          object        
 6   company_enter_month                  object        
 7   employment_type                      object        
 8   houseown_type                        object        
 9   desired_amount                       float64       
 10  purpose                              object        
 11  personal_rehabilitation_yn           float64       
 12  personal_rehabilitation_complete_yn  object        
 13  existing_loan_cnt          

## loan 파생변수 파일 만들기

In [ ]:
# 은행 별 상품 추천 수, 상품 수, 대출 승인한 확률
count_bank=train_result.groupby('bank_id').agg(**{
    'bank_count':pd.NamedAgg(column='bank_id', aggfunc='count'),
    'bank_product':pd.NamedAgg(column='product_id', aggfunc=lambda x:x.nunique()),
    'apply_sum':pd.NamedAgg(column='is_applied', aggfunc='sum')
}).reset_index()

In [ ]:
# 은행 별 승인한 확률
count_bank['apply_rate_bank'] = round(count_bank['apply_sum']/count_bank['bank_count'],4)

In [ ]:
# 은행 별 승인한 확률 평균
app_mean = round(train_result['is_applied'].sum()/train_result['is_applied'].count(),3)
app_mean

In [ ]:
# 10보다 작은 수는 불균등하기에, 평규으로 대체
count_bank.loc[count_bank['bank_count']<10,'apply_rate_bank']=app_mean

In [ ]:
count_bank.drop(['apply_sum'], axis=1, inplace=True)

In [ ]:
count_bank

In [ ]:
# 상품 별 추천 수, 대출 승인한 확률
count_pro=train.groupby('product_id').agg(**{
    'product_count':pd.NamedAgg(column='product_id', aggfunc='count'),
    'apply_sum':pd.NamedAgg(column='is_applied', aggfunc='sum')
}).reset_index()

In [ ]:
# 승인한 확률
count_pro['pro_apply_rate'] = round(count_pro['apply_sum']/count_pro['product_count'],4)

In [ ]:
# 10보다 작은 수는 불균등하기에, 평규으로 대체
count_pro.loc[count_pro['product_count']<=2,'pro_apply_rate']=app_mean

In [ ]:
count_pro.drop(['apply_sum'], axis=1, inplace=True)

In [ ]:
count_pro

In [ ]:
# 파일내보내기
count_bank.to_csv('./count_bank.csv', encoding = 'cp949')

In [ ]:
count_pro.to_csv('./count_prod.csv', encoding = 'cp949')